In [ ]:
%reset

In [ ]:
import os, sys

import pandas as pd
import numpy as np
import swifter

import pacmagic
import pacmagic_icu

from importlib import reload
from data_warehouse_utils.dataloader import DataLoader

os.chdir('/home/adam/files/causal_inference')
os.getcwd()

from causal_inference.experiment_generator.create_data_points import create_data_points

In [ ]:
# Reloads packages

reload(sys.modules['causal_inference'])
reload(sys.modules['causal_inference.experiment_generator'])
reload(sys.modules['causal_inference.experiment_generator.initialize_experiment'])
reload(sys.modules['causal_inference.experiment_generator.create_data_points'])

In [ ]:
dl = DataLoader()

In [ ]:
df = create_data_points(dl, 10)

In [ ]:
df.head(10)

In [ ]:
df.info()

In [ ]:
df[~df.po2_arterial.isna()]

In [ ]:
# Should just check if it is intubated for each patient.

df.dropna(axis='index', how='any')